# Prompt Engineering with Amazon Bedrock

In this notebook, you explore different prompt engineering techniques. A *prompt* is the message sent to a model to generate a response. Prompt engineering might involve:
* Word choice
* Phrasing
* Providing additional information
* Providing examples

1. **Generate recommendations based on metadata.**
    - **Task:** Text generation
    - **Prompt technique:** Zero-shot
2. **Estimate audience for TV shows based on historical data.**
    - **Task:** Complex reasoning
    - **Prompt technique:** Chain-of-thought (CoT)
3. **Create a question-answering assistant.**
    - **Task:** Question answering with a dialogue assistant (without memory)
    - **Prompt technique:** Few-shot
4. **Create splash pages that describe upcoming events.**
    - **Task:** Code generation
    - **Prompt technique:** Zero-shot

### Import libraries and create an Amazon Bedrock client.

First, import the needed libraries and create an Amazon Bedrock Boto client. Then, list the available models from Amazon Bedrock.

In [ ]:
## Code Cell 1 ##

import boto3
import json
import csv
from datetime import datetime


model_id = "amazon.nova-pro-v1:0"
bedrock = boto3.client('bedrock')
bedrock_runtime = boto3.client('bedrock-runtime')

### Create a help function for calling Amazon Bedrock. 
Define a utility function for calling Amazon Bedrock to help pass the proper body, depending on the invoked model.

In [ ]:
## Code Cell 2 ##

def call_bedrock(modelId, prompt_data):
    body = json.dumps({
    "inferenceConfig": {
      "max_new_tokens": 1000
    },
    "messages": [
      {
        "role": "user",
        "content": [
          {
            "text": prompt_data
          }
        ]
      }
    ]
  })
    accept = 'application/json'
    contentType = 'application/json'

    before = datetime.now()
    response = bedrock_runtime.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
    latency = (datetime.now() - before)
    response_body = json.loads(response.get('body').read())

    response = response_body.get('output').get('message').get('content')[0].get('text')

    return response, latency



Now, you're ready to run examples with different prompt engineering techniques. 

-----

## 1. Generate recommendations based on metadata.

**Use case:** A media and entertainment company wants to generate TV show recommendations for their customers based on viewer metadata, sucha as country, age range, and theme.

**Task:** Text generation

**Prompt technique:** Zero-shot

A single prompt is comprised of a number of different components, wnich might include:
* A task or instruction
* Context of a task, such as a description of the relevant domain
* Demonstration examples
* Input text, which you want the model to use in forming a response

Your prompt will be a combination of one or more of these components, depending on the use case, the availability of the data, and the task.

In [ ]:
## Code Cell 3 ##

prompt_data ="""
Generate a list of 10 recommended TV shows to watch, considering the information in the <metadata></metadata> XML tags. Include a very brief description of each recommendation.

<metadata>
Country is UK
Age range between 20-30
Shows must be about sports
</metadata>

"""


response, latency = call_bedrock(model_id, prompt_data)
print(response, "\n\n", "Inference time:", latency)

------

## 2. Estimate audience for TV shows based on historical data.

**Use case:** The company wants to estimate the next day's audience levels based on historical information and a TV show's metadata.

**Task:** Complex reasoning

**Prompt technique:** Chain-of-thought (CoT)

In [ ]:
## Code Cell 4 ##

prompt_data ="""
- Monday: SportsTV 6500, NewsTV 3200, EntertainmentTV 4150
- Tuesday: SportsTV 6400, NewsTV 3300, EntertainmentTV 4100
- Wednesday: SportsTV 6300, NewsTV 3400, EntertainmentTV 4250

Question: How many viewers can we expect next Friday on SportsTV?
Answer: According to the numbers given, and without having more information, there is a daily decrease of 100 viewers for SportsTV.
If we assume that this trend will continue for the next few days, we can expect 6200 viewers for the next day, which is Thursday, and
therefore 6100 viewers for Friday.

Question: How many viewers can we expect on Saturday for each of the three channels? Think step-by-step, and provide recommendations for increasing the viewers.
"""

response, latency = call_bedrock(model_id, prompt_data)
print(response, "\n\n", "Inference time:", latency)

------

## 3. Create a question-answering assistant.

**Use case:** The company wants to create a dialogue assistant that is capable of answering questions about available TV shows based on show data.

**Task:** Question answering with a dialogue Assistant (no memory)

**Prompt technique:** Few-shot

In [ ]:
## Code Cell 5 ##

prompt_data ="""
Context: The shows available are as follows
1. Circus, showing at the Plaza venue, assigned seating, live at 8pm on weekends
2. Concert, showing at the Main Theater, assigned seating, live at 10pm everyday
3. Basketball tricks, showing at the Sports venue, standing seating, live at 5pm on weekdays

Instruction: Answer any questions about the available shows. If you don't know the answer, say 'Apologies, I don't have the answer for that. Please contact our team by phone.'

Assistant: Welcome to Entertainment Tonight, how can I help you?
Human: Hi, I would like to know what shows are available please.
Assistant: Of course. Right now, we have the Circus, the Concert, and the Basketball tricks shows.
Human: Thank you. I would like to know when and where those shows are available please.
Assistant:
"""

response, latency = call_bedrock(model_id, prompt_data)
print(response, "\n\n", "Inference time:", latency)

------

## 4. Create splash pages that describe upcoming events.

**Use case:** The company wants a quick, efficient way to create HTML pages to promote upcoming events.

**Task:** Code generation

**Prompt technique:** Zero-shot

In [ ]:
## Code Cell 6 ##

prompt_data ="""
An upcoming music concert is presented by the company, Music Promotions.
The event targets a young audience, age range between 18 and 40.
The event will occur in the Royal Music Theater.
Seating is assigned and tickets can be purchased through the Music Promotions website.
The event is a music concert performed by the band, Super Rockers.
The event will occur on June 30, 2023, and doors will open at 20:00.

Based on the information provided above, generate the HTML code for an attractive splash page to promote the event.
"""

response, latency = call_bedrock(model_id, prompt_data)
print(response, "\n\n", "Inference time:", latency)
from IPython.display import display, HTML
display(HTML(response))


------
## DIY – Use a prompt template to improve the application prompt. 
------

1. Use the Code Cell 7 call_bedrock Python function to update the invoke_bedrock Lambda function. 
2. After updating the call_bedrock function, redeploy the function by using the same steps that you used during the Practice section. 
3. Test the application again, asking for a TV show recommendation, to see how the model response improved with the user metadata.


In [ ]:
# Code Cell 7

def call_bedrock(modelId, prompt_data):
    bedrock_runtime = boto3.client('bedrock-runtime')
    
    prompt_template = """
    User Metadata:
    Name: John Doe
    Age: 35
    Location: New York, USA
    Interests: Technology, Artificial Intelligence, Music
    Occupation: Software Engineer
    
    Question: {prompt_data}
    """.format(prompt_data=prompt_data)
    
    body = json.dumps({
    "inferenceConfig": {
      "max_new_tokens": 1000
    },
    "messages": [
      {
        "role": "user",
        "content": [
          {
            "text": prompt_template
          }
        ]
      }
    ]
  })
    print("bedrock-input:",body)

    accept = 'application/json'
    contentType = 'application/json'

    before = datetime.now()
    response = bedrock_runtime.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
    latency = (datetime.now() - before).seconds
    response_body = json.loads(response.get('body').read())
    response = response_body.get('output').get('message').get('content')[0].get('text')    

    return {
        'latency': str(latency),
        'response': response
    }